In [ ]:
import datetime
now = datetime.datetime.now()
print(now)
nowDate = now.strftime('%Y/%m/%d')
print("today = ", nowDate)

In [ ]:
from datetime import date,timedelta

def findLastSunOrWed():
    targetDay = date.today()
    
    # weekday 2 = Wed 
    # weekday 6 = Sun
    offsetWed = (targetDay.weekday() - 2) % 7
    offsetSun = (targetDay.weekday() - 6) % 7
    offset = min(offsetWed, offsetSun)
    
    targetDay = targetDay - timedelta(days=offset)
    #print("Target Day : ", targetDay, targetDay.weekday())
    
    return targetDay.strftime('%Y/%m/%d')

In [ ]:
folder = findLastSunOrWed()
print("target =", folder)

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
from crawMatch import *

In [ ]:
clickTabOfFireFox(4)
clickEmpty(79/300)
moveScroll(20)
clickLogin()
clickMyClub()
clickEmpty(190/300)
moveScroll(20)
clickMatch()
clickLastMatch()
clickFirefoxFile()
clickSavePageAs()
typeFileName()
clickHome()
click02_workspace()
click02_Hattrick()
clickHattrickStudy()
click01_Heavy2Defence()
click2017()
clickLastMonth()
clickLastDay()
clickSave()
clickLogout()
clickURL()
clickTabOfFireFox(7)

In [ ]:
import os.path, shutil
filePath = folder + '/match_files'

if ( os.path.exists(filePath) ):
    shutil.rmtree(filePath, ignore_errors=True)
    print(filePath + " <- this folder is deleted.")

if not ( os.path.exists(filePath) ):
    print(filePath + " <- this folder is not exists.")

In [ ]:
with open(folder+'/match.html', 'r') as file:
    soup = BeautifulSoup(file.read(), "html.parser")

In [ ]:
from parseMatch import *

In [ ]:
import os.path
filePath = folder + '/match.txt'

if ( os.path.exists(filePath) ):
    os.remove(filePath)
    print(filePath + " <- this file is deleted.")

if not ( os.path.exists(filePath) ):
    print(filePath + " <- this file is not exists.")

In [ ]:
for div in soup.find_all("div", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_timelineEventPanel"):
    for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsAway"):
    #for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsHome"):
        #print(inputTag.prettify())
        valueString = inputTag.get('value')
        valueString = re.sub('},', '}\n', valueString)
        
        matchList = valueString.split("\n")            
        matchList = removeFirstChrLastChr(matchList)
        matchList = filterColumn(matchList)
        matchList = renamePO(matchList)
        matchList = renameMin(matchList)
        matchList = changedPlayer(matchList)
        matchList = renameNumber(matchList, folder)
        matchList = sortByPO(matchList)
        matchList = removeRows(matchList)
        
        print("\ncat match.txt\n")
        
        for line in matchList:
            print(line)

In [ ]:
import os.path
filePath = folder + '/match.txt'

if ( os.path.exists(filePath) ):
    os.remove(filePath)
    print(filePath + " <- this file is deleted.")

if not ( os.path.exists(filePath) ):
    print(filePath + " <- this file is not exists.")

In [ ]:
for line in matchList:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [ ]:
# From match.txt to match table

In [ ]:
import psycopg2

In [ ]:
def dropMatchTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """DROP TABLE IF EXISTS match ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
def createMatchTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
CREATE TABLE match (
    Date DATE,          --01
    Po   VARCHAR(5),    --02 Position
    Num  INT,           --03 Number
    RT   NUMERIC(2, 1), --04 Rating
    sMin INT,           --05 Start Min
    eMin INT            --06 End Min
) ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#createMatchTable()

In [ ]:
def selectCountPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = "SELECT COUNT(*) FROM match" 
        
        cur.execute(sql)
        row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()
    return row[0]

In [ ]:
selectCountPlayer()

In [ ]:
def insertMatchTable(time, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
INSERT INTO match ( 
    SELECT 
        to_date(%s, 'YYYY-MM-DD'), --01 
        %s, %s, %s, %s, %s         --02 03 04 05 06
) ;"""
        cur.execute(sql, ( time,
                           row[0], row[1], row[2], row[3], row[4] ) )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
        print(row)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            insertMatchTable( folder.replace('/','-'), row )

In [ ]:
selectCountPlayer()

In [ ]:
def updateRatingOfMatchTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
update match 
    set rt = player.rt 
from player   
where 
    match.date = player.date 
    and player.date = player.last 
    and match.po = player.po 
    and match.num = player.num 
    and match.rt != player.rt
; """
        cur.execute(sql)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
updateRatingOfMatchTable()

In [ ]:
def getTupleListFromMatch(query):
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
query = ""
query += "\n" + "SELECT "
query += "\n" + "    po, num, rt :: VARCHAR(5), smin, emin "
query += "\n" + "FROM match "
query += "\n" + "WHERE "
query += "\n" + "    date ='" + folder.replace('/','-') + "'"
query += "\n" + "ORDER BY "
query += "\n" + "    CASE "
query += "\n" + "        WHEN po = 'KP' THEN 1"
query += "\n" + "        WHEN po = 'WB' THEN 2"
query += "\n" + "        WHEN po = 'CD' THEN 3"
query += "\n" + "        WHEN po = 'W'  THEN 4"
query += "\n" + "        WHEN po = 'IM' THEN 5"
query += "\n" + "        WHEN po = 'FW' THEN 6"
query += "\n" + "        ELSE 7 "
query += "\n" + "    END, "
query += "\n" + "    num "

tupleList = getTupleListFromMatch(query)
for tuple in tupleList:
    print(tuple[0], tuple[1], tuple[2], tuple[3], tuple[4], sep='\t')

In [ ]:
import os.path
filePath = folder + '/match.txt'

if ( os.path.exists(filePath) ):
    os.remove(filePath)
    print(filePath + " <- this file is deleted.")

if not ( os.path.exists(filePath) ):
    print(filePath + " <- this file is not exists.")

In [ ]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [ ]:
def getKP():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =         """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''KP'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date,num', """
        query += "\n" + """    'SELECT m from unnest(''{1,2}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   1   " VARCHAR(7), """
        query += "\n" + """    "   2   " VARCHAR(7) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
def xstr(s):
    if s is None:
        return '       '
    else :
        if len(s) == 6:
            s = ' ' + s
        if len(s) == 5:
            s = '  ' + s
    return str(s)

In [ ]:
tupleList = getKP()
print('date num min/rt |    01   |     02   ')
print('----------------+---------+----------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), sep=" | ")

In [ ]:
def getWB():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =         """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''WB'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date, num', """
        query += "\n" + """    'SELECT m from unnest(''{4,13,14,15}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   4   " VARCHAR(8), """
        query += "\n" + """    "   13   " VARCHAR(8), """
        query += "\n" + """    "   14  " VARCHAR(8), """
        query += "\n" + """    "   15  " VARCHAR(8) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getWB()
print('date num min/rt |    4    |    13   |    14   |    15   ')
print('----------------+---------+---------+---------+---------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), sep=" | ")

In [ ]:
def getCD():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =  """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''CD'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date, num', """
        query += "\n" + """    'SELECT m from unnest(''{6,8,9,10,11,33}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   6   " VARCHAR(8), """
        query += "\n" + """    "   8   " VARCHAR(8), """
        query += "\n" + """    "   9   " VARCHAR(8), """
        query += "\n" + """    "   10  " VARCHAR(8), """
        query += "\n" + """    "   11  " VARCHAR(8), """
        query += "\n" + """    "   33  " VARCHAR(8) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getCD()
print('date num min/rt |    6    |    8    |    9    |    10   |    11   |    33  ')
print('----------------+---------+---------+---------+---------+---------+--------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), xstr(tuple[5]), xstr(tuple[6]),sep=" | ")

In [ ]:
def getW():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =  """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''W'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date, num', """
        query += "\n" + """    'SELECT m from unnest(''{3,5,7,28,35}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   3   " VARCHAR(8), """
        query += "\n" + """    "   5   " VARCHAR(8), """
        query += "\n" + """    "   7   " VARCHAR(8), """
        query += "\n" + """    "   28  " VARCHAR(8), """
        query += "\n" + """    "   35  " VARCHAR(8) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getW()
print('date num min/rt |    3    |    5    |    7    |    28   |    35   ')
print('----------------+---------+---------+---------+---------+---------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), xstr(tuple[5]), sep=" | ")

In [ ]:
def getIM():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =  """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''IM'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date, num', """
        query += "\n" + """    'SELECT m from unnest(''{12,18,29,31}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   12  " VARCHAR(8), """
        query += "\n" + """    "   18  " VARCHAR(8), """
        query += "\n" + """    "   29  " VARCHAR(8), """
        query += "\n" + """    "   31  " VARCHAR(8) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getIM()
print('date num min/rt |    12   |    18   |    29   |    31   ')
print('----------------+---------+---------+---------+---------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), sep=" | ")

In [ ]:
def getFW():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        query =  """SELECT * """
        query += "\n" + """FROM crosstab( """
        query += "\n" + """    'SELECT """
        query += "\n" + """        date :: VARCHAR(10), num, emin-smin||''/''||rt :: VARCHAR(7) as time_rt """
        query += "\n" + """    FROM match """
        query += "\n" + """    WHERE """
        query += "\n" + """        po = ''FW'' """
        query += "\n" + """    ORDER BY """
        query += "\n" + """        date, num', """
        query += "\n" + """    'SELECT m from unnest(''{16,17,33}''::int[]) m' """
        query += "\n" + """) """
        query += "\n" + """AS ( """
        query += "\n" + """    "date num min/rt" VARCHAR(10), """
        query += "\n" + """    "   16  " VARCHAR(8), """
        query += "\n" + """    "   17  " VARCHAR(8), """
        query += "\n" + """    "   33  " VARCHAR(8) """
        query += "\n" + """)"""
        query += "\n" + """ORDER BY 1"""

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getFW()
print('date num min/rt |    16   |    17   |    33   ')
print('----------------+---------+---------+---------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ',xstr(tuple[1]),xstr(tuple[2]),xstr(tuple[3]), sep=" | ")